In [1]:
import pandas as pd
import numpy as np
import json
from pandas.io.json import json_normalize
import matplotlib.cm as cm
import matplotlib.colors as color

# !conda install -c conda-forge geopy --yes
from geopy.geocoders import Nominatim

from sklearn.cluster import KMeans
import matplotlib.cm as cm
import matplotlib.colors as colors

!conda install -c conda-forge folium=0.5.0 --yes
import folium

Solving environment: - ^C
failed

CondaError: KeyboardInterrupt



In [44]:
# The code was removed by Watson Studio for sharing.

Your credentails:
CLIENT_ID: I0I2Z1LMEYME25RRI1SZ5IWWA1VLUDTFV33GWNRQ2RUSJGLV
CLIENT_SECRET:5EMQGIQL3CXG4XG0R0LFMH2TJM1ISA241IENYIMOEI5VNZDG


In [3]:
address = input("enter the name of city ")
geolocator = Nominatim(user_agent = "{}_explorer".format(address))

location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude

print(latitude,longitude)

enter the name of city washington
38.8949855 -77.0365708


In [45]:
# The code was removed by Watson Studio for sharing.

'https://api.foursquare.com/v2/venues/explore?&client_id=I0I2Z1LMEYME25RRI1SZ5IWWA1VLUDTFV33GWNRQ2RUSJGLV&client_secret=5EMQGIQL3CXG4XG0R0LFMH2TJM1ISA241IENYIMOEI5VNZDG&v=20180605&ll=38.8949855,-77.0365708&query=tourist&radius=50000&limit=10000'

In [5]:
import requests
results = requests.get(url).json()
results

{'meta': {'code': 200, 'requestId': '5f2bda9bcf2d854abdfc479c'},
 'response': {'suggestedFilters': {'header': 'Tap to show:',
   'filters': [{'name': 'Open now', 'key': 'openNow'},
    {'name': '$-$$$$', 'key': 'price'}]},
  'headerLocation': 'Washington',
  'headerFullLocation': 'Washington',
  'headerLocationGranularity': 'city',
  'query': 'tourist',
  'totalResults': 250,
  'suggestedBounds': {'ne': {'lat': 39.34498595000045,
    'lng': -76.459465131844},
   'sw': {'lat': 38.444985049999545, 'lng': -77.61367646815602}},
  'groups': [{'type': 'Recommended Places',
    'name': 'recommended',
    'items': [{'reasons': {'count': 0,
       'items': [{'summary': 'This spot is popular',
         'type': 'general',
         'reasonName': 'globalInteractionReason'}]},
      'venue': {'id': '4c6199b5edd320a12defaa29',
       'name': 'National Museum of African American History and Culture',
       'location': {'address': '1400 Constitution Ave NW',
        'crossStreet': 'btwn 12th & 14th St

In [46]:
# The code was removed by Watson Studio for sharing.

'https://api.foursquare.com/v2/venues/explore?&client_id=I0I2Z1LMEYME25RRI1SZ5IWWA1VLUDTFV33GWNRQ2RUSJGLV&client_secret=5EMQGIQL3CXG4XG0R0LFMH2TJM1ISA241IENYIMOEI5VNZDG&v=20180605&ll=38.8949855,-77.0365708&query=tourist&radius=50000&limit=10000'

In [7]:
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

In [8]:
venues = results['response']['groups'][0]['items']
    
nearby_venues = json_normalize(venues) # flatten JSON

# filter columns
filtered_columns = ['venue.id','venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
nearby_venues =nearby_venues.loc[:, filtered_columns]

# filter the category for each row
nearby_venues['venue.categories'] = nearby_venues.apply(get_category_type, axis=1)

# clean columns
nearby_venues.columns = [col.split(".")[-1] for col in nearby_venues.columns]

nearby_venues.head()

,id,name,categories,lat,lng
0,4c6199b5edd320a12defaa29,National Museum of African American History an...,History Museum,38.891171,-77.032818
1,40ce3c00f964a5201b011fe3,Washington Monument,Monument / Landmark,38.889401,-77.035244
2,4aa978f7f964a520d75320e3,The Willard InterContinental Washington D.C. H...,Hotel,38.897215,-77.032541
3,4b75a2b7f964a520c2182ee3,The Hay-Adams,Hotel,38.900510,-77.036885
4,4a518853f964a5201bb11fe3,World War II Memorial,Monument / Landmark,38.889377,-77.040516


# User data for venues which they have already visited

In [9]:
user_input = [{'venue_name' : 'Eiffel Tower', 'rating': '6.4'},
             {'venue_name' : 'Taj Mahal', 'rating': '6.4'},
             {'venue_name' : 'Dal Lake', 'rating': '8.9'}
             ]

In [10]:
user_input = pd.DataFrame(user_input)

In [11]:
user_input

,rating,venue_name
0,6.4,Eiffel Tower
1,6.4,Taj Mahal
2,8.9,Dal Lake


# finding the latitude and longitude of venues present in 'user input' dataframe

In [12]:
lat =[]
lng = []
for ven in user_input['venue_name']:
    # address = "washington"
    geolocator = Nominatim(user_agent = "{}_explorer".format(ven))

    location = geolocator.geocode(ven)
    latitude = location.latitude
    longitude = location.longitude    
    lat.append(latitude)
    lng.append(longitude)
    
user_input['lat'] = pd.Series(lat)
user_input['lng'] = pd.Series(lng)
user_input


,rating,venue_name,lat,lng
0,6.4,Eiffel Tower,48.858260,2.294499
1,6.4,Taj Mahal,27.175012,78.042097
2,8.9,Dal Lake,34.113678,74.869991


# finding all the details of venues in 'user_input' with the help of foursquare api

In [13]:
user_data = pd.DataFrame(None)
user_ven = pd.DataFrame(None)
for i,ven in zip(range(0,3),user_input['venue_name']):
    latitude = user_input.loc[i,'lat']
    longitude = user_input.loc[i,'lng']
    LIMIT=1
    url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&query={}&radius={}&limit={}'.format(CLIENT_ID, CLIENT_SECRET, VERSION, latitude, longitude, ven, RADIUS, LIMIT)
    res =  requests.get(url).json()
    venues = res['response']['groups'][0]['items']
    
    
    user_ven = json_normalize(venues) # flatten JSON
    

 # filter columns
    filtered_columns = ['venue.id','venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
    user_ven = user_ven.loc[:, filtered_columns]
    
    
# # filter the category for each row
    user_ven['venue.categories'] = user_ven.apply(get_category_type, axis=1)
   
 # clean columns
    user_data = user_data.append(user_ven)

user_data

,venue.id,venue.name,venue.categories,venue.location.lat,venue.location.lng
0,51a2445e5019c80b56934c75,Tour Eiffel,Monument / Landmark,48.858258,2.294423
0,4b53fce2f964a52014b027e3,Taj Mahal | ताज महल | تاج محل (Taj Mahal),Historic Site,27.171047,78.042187
0,4bcf326e77b29c74c6968882,Dal Lake | डल झील | دل ليك,Lake,34.087640,74.856892


In [14]:
user_data.columns = [col.split(".")[-1] for col in user_data.columns]

In [15]:
user_data = user_data.reset_index()
user_data = user_data.drop('index',1)

In [16]:
user_data

,id,name,categories,lat,lng
0,51a2445e5019c80b56934c75,Tour Eiffel,Monument / Landmark,48.858258,2.294423
1,4b53fce2f964a52014b027e3,Taj Mahal | ताज महल | تاج محل (Taj Mahal),Historic Site,27.171047,78.042187
2,4bcf326e77b29c74c6968882,Dal Lake | डल झील | دل ليك,Lake,34.087640,74.856892


# making a global dataframe by concating the 'nearby_venues' and 'user_data' table which will help in developing a recomendation system 

In [17]:
venues_df = pd.concat([nearby_venues, user_data], axis=0) 

In [18]:
venues_df.head()

,id,name,categories,lat,lng
0,4c6199b5edd320a12defaa29,National Museum of African American History an...,History Museum,38.891171,-77.032818
1,40ce3c00f964a5201b011fe3,Washington Monument,Monument / Landmark,38.889401,-77.035244
2,4aa978f7f964a520d75320e3,The Willard InterContinental Washington D.C. H...,Hotel,38.897215,-77.032541
3,4b75a2b7f964a520c2182ee3,The Hay-Adams,Hotel,38.900510,-77.036885
4,4a518853f964a5201bb11fe3,World War II Memorial,Monument / Landmark,38.889377,-77.040516


In [19]:
venues_df = venues_df.reset_index()
venues_df = venues_df.drop('index',1)

In [20]:
venues_df.head()

,id,name,categories,lat,lng
0,4c6199b5edd320a12defaa29,National Museum of African American History an...,History Museum,38.891171,-77.032818
1,40ce3c00f964a5201b011fe3,Washington Monument,Monument / Landmark,38.889401,-77.035244
2,4aa978f7f964a520d75320e3,The Willard InterContinental Washington D.C. H...,Hotel,38.897215,-77.032541
3,4b75a2b7f964a520c2182ee3,The Hay-Adams,Hotel,38.900510,-77.036885
4,4a518853f964a5201bb11fe3,World War II Memorial,Monument / Landmark,38.889377,-77.040516


In [21]:
print('There are {} uniques categories.'.format(len(venues_df['categories'].unique())))

There are 54 uniques categories.


# Building a Recommendation System for recommending the venues to the user

# calculating weight for user table to build a recommendation system

In [22]:
venues_df['categories'] = venues_df.categories.str.split('/')
venues = venues_df.copy()

for index, row in venues_df.iterrows():
    for category in row['categories']:
        venues.at[index,category] = 1
    venues = venues.fillna(0)
venues.head()

,id,name,categories,lat,lng,History Museum,Monument,Landmark,Hotel,American Restaurant,...,Bar,Beer Bar,Sushi Restaurant,Bakery,Beer Garden,Office,Grocery Store,Library,Whisky Bar,Historic Site
0,4c6199b5edd320a12defaa29,National Museum of African American History an...,[History Museum],38.891171,-77.032818,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,40ce3c00f964a5201b011fe3,Washington Monument,"[Monument , Landmark]",38.889401,-77.035244,0.0,1.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,4aa978f7f964a520d75320e3,The Willard InterContinental Washington D.C. H...,[Hotel],38.897215,-77.032541,0.0,0.0,0.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,4b75a2b7f964a520c2182ee3,The Hay-Adams,[Hotel],38.900510,-77.036885,0.0,0.0,0.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,4a518853f964a5201bb11fe3,World War II Memorial,"[Monument , Landmark]",38.889377,-77.040516,0.0,1.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [23]:
user_detail = venues[venues['id'].isin(user_data['id'].tolist())]

In [24]:
user_detail = user_detail.reset_index(drop=True)

In [25]:
user_detail_table = user_detail.drop('id', 1).drop('name',1).drop('lat',1).drop('lng', 1).drop('categories',1)

In [26]:
user_detail_table

,History Museum,Monument,Landmark,Hotel,American Restaurant,Seafood Restaurant,Government Building,Garden,Science Museum,Art Museum,...,Bar,Beer Bar,Sushi Restaurant,Bakery,Beer Garden,Office,Grocery Store,Library,Whisky Bar,Historic Site
0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [27]:
user_input['rating'] = pd.to_numeric(user_input['rating'], downcast ='float')
userProfile = user_detail_table.transpose().dot(user_input['rating'])
userProfile

History Museum                 0.0
Monument                       6.4
 Landmark                      6.4
Hotel                          0.0
American Restaurant            0.0
Seafood Restaurant             0.0
Government Building            0.0
Garden                         0.0
Science Museum                 0.0
Art Museum                     0.0
Plaza                          0.0
Lake                           8.9
Ice Cream Shop                 0.0
Sculpture Garden               0.0
Indian Restaurant              0.0
Poke Place                     0.0
Performing Arts Venue          0.0
Electronics Store              0.0
Mediterranean Restaurant       0.0
Brazilian Restaurant           0.0
Coffee Shop                    0.0
Park                           0.0
Burger Joint                   0.0
Cocktail Bar                   0.0
Outdoor Sculpture              0.0
Latin American Restaurant      0.0
Café                           0.0
Italian Restaurant             0.0
Trail               

In [28]:
category_table = venues.set_index(venues['id'])
category_table.head()

,id,name,categories,lat,lng,History Museum,Monument,Landmark,Hotel,American Restaurant,...,Bar,Beer Bar,Sushi Restaurant,Bakery,Beer Garden,Office,Grocery Store,Library,Whisky Bar,Historic Site
id,,,,,,,,,,,,,,,,,,,,,
4c6199b5edd320a12defaa29,4c6199b5edd320a12defaa29,National Museum of African American History an...,[History Museum],38.891171,-77.032818,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
40ce3c00f964a5201b011fe3,40ce3c00f964a5201b011fe3,Washington Monument,"[Monument , Landmark]",38.889401,-77.035244,0.0,1.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4aa978f7f964a520d75320e3,4aa978f7f964a520d75320e3,The Willard InterContinental Washington D.C. H...,[Hotel],38.897215,-77.032541,0.0,0.0,0.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4b75a2b7f964a520c2182ee3,4b75a2b7f964a520c2182ee3,The Hay-Adams,[Hotel],38.900510,-77.036885,0.0,0.0,0.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4a518853f964a5201bb11fe3,4a518853f964a5201bb11fe3,World War II Memorial,"[Monument , Landmark]",38.889377,-77.040516,0.0,1.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


# RECOMMENDATION ENGINE FOR SUGGESTING PLACES

# user details that consist venues where they vists before 

In [29]:
category_table = category_table.drop('id',1).drop('name',1).drop('categories',1).drop('lat',1).drop('lng',1)

In [30]:
category_table.head()

,History Museum,Monument,Landmark,Hotel,American Restaurant,Seafood Restaurant,Government Building,Garden,Science Museum,Art Museum,...,Bar,Beer Bar,Sushi Restaurant,Bakery,Beer Garden,Office,Grocery Store,Library,Whisky Bar,Historic Site
id,,,,,,,,,,,,,,,,,,,,,
4c6199b5edd320a12defaa29,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
40ce3c00f964a5201b011fe3,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4aa978f7f964a520d75320e3,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4b75a2b7f964a520c2182ee3,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4a518853f964a5201bb11fe3,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [31]:
recommendation_df = ((category_table * userProfile).sum(axis = 1))/(userProfile.sum())
recommendation_df = recommendation_df.sort_values(ascending = False)
recommendation_df.head(15)

id
4a500fe1f964a520f2af1fe3    0.455516
51a2445e5019c80b56934c75    0.455516
40ce3c00f964a5201b011fe3    0.455516
4a10649df964a520b7761fe3    0.455516
4a491e86f964a520ffaa1fe3    0.455516
4a518853f964a5201bb11fe3    0.455516
4a106621f964a520ba761fe3    0.455516
4bc9f0850687ef3bc7d0dacc    0.455516
4bc4fdf20969a5931053636e    0.455516
4b65f636f964a5203e0b2be3    0.455516
4a4fbaecf964a520a0af1fe3    0.455516
4bcf326e77b29c74c6968882    0.316726
49d69653f964a520ed5c1fe3    0.316726
4b53fce2f964a52014b027e3    0.227758
3fd66200f964a520d6f01ee3    0.000000
dtype: float64

In [32]:
recommended_venues = nearby_venues.loc[nearby_venues['id'].isin(recommendation_df.head(14).keys())]

In [33]:
recommended_venues

,id,name,categories,lat,lng
1,40ce3c00f964a5201b011fe3,Washington Monument,Monument / Landmark,38.889401,-77.035244
4,4a518853f964a5201bb11fe3,World War II Memorial,Monument / Landmark,38.889377,-77.040516
12,4a500fe1f964a520f2af1fe3,Vietnam Veterans Memorial,Monument / Landmark,38.891262,-77.047707
14,4a4fbaecf964a520a0af1fe3,Lincoln Memorial,Monument / Landmark,38.889267,-77.050166
16,4b65f636f964a5203e0b2be3,Lincoln Memorial Reflecting Pool,Monument / Landmark,38.889491,-77.045649
20,4bc9f0850687ef3bc7d0dacc,"Martin Luther King, Jr. Memorial",Monument / Landmark,38.886087,-77.044124
23,49d69653f964a520ed5c1fe3,Tidal Basin,Lake,38.884193,-77.039314
30,4a106621f964a520ba761fe3,Thomas Jefferson Memorial,Monument / Landmark,38.881712,-77.036654
32,4a491e86f964a520ffaa1fe3,Korean War Veterans Memorial,Monument / Landmark,38.887790,-77.047472
35,4a10649df964a520b7761fe3,Franklin Delano Roosevelt Memorial,Monument / Landmark,38.883409,-77.042788


# onehot encoding


In [34]:
recommended_onehot = pd.get_dummies(recommended_venues[['name']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
recommended_onehot['categories'] = recommended_venues['categories'] 

# move neighborhood column to the first column
fixed_columns = [recommended_onehot.columns[-1]] + list(recommended_onehot.columns[:-1])
recommended_onehot = recommended_onehot[fixed_columns]

recommended_onehot.head()

,categories,Capitol Reflecting Pool,Franklin Delano Roosevelt Memorial,Korean War Veterans Memorial,Lincoln Memorial,Lincoln Memorial Reflecting Pool,"Martin Luther King, Jr. Memorial",Thomas Jefferson Memorial,Tidal Basin,Vietnam Veterans Memorial,Washington Monument,World War II Memorial
1,Monument / Landmark,0,0,0,0,0,0,0,0,0,1,0
4,Monument / Landmark,0,0,0,0,0,0,0,0,0,0,1
12,Monument / Landmark,0,0,0,0,0,0,0,0,1,0,0
14,Monument / Landmark,0,0,0,1,0,0,0,0,0,0,0
16,Monument / Landmark,0,0,0,0,1,0,0,0,0,0,0


In [35]:
recommended_group = recommended_onehot.groupby('categories').mean().reset_index()
recommended_group

,categories,Capitol Reflecting Pool,Franklin Delano Roosevelt Memorial,Korean War Veterans Memorial,Lincoln Memorial,Lincoln Memorial Reflecting Pool,"Martin Luther King, Jr. Memorial",Thomas Jefferson Memorial,Tidal Basin,Vietnam Veterans Memorial,Washington Monument,World War II Memorial
0,Lake,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
1,Monument / Landmark,0.1,0.1,0.1,0.1,0.1,0.1,0.1,0.0,0.1,0.1,0.1


In [36]:
num_top_venues = 5

for category in recommended_group['categories']:
    print("----"+category+"----")
    temp = recommended_group[recommended_group['categories'] == category].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Lake----
                                venue  freq
0                         Tidal Basin   1.0
1             Capitol Reflecting Pool   0.0
2  Franklin Delano Roosevelt Memorial   0.0
3        Korean War Veterans Memorial   0.0
4                    Lincoln Memorial   0.0


----Monument / Landmark----
                                venue  freq
0             Capitol Reflecting Pool   0.1
1  Franklin Delano Roosevelt Memorial   0.1
2        Korean War Veterans Memorial   0.1
3                    Lincoln Memorial   0.1
4    Lincoln Memorial Reflecting Pool   0.1




In [37]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [38]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['categories']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
recommended_venues_sorted = pd.DataFrame(columns=columns)
recommended_venues_sorted['categories'] = recommended_group['categories']

for ind in np.arange(recommended_group.shape[0]):
    recommended_venues_sorted.iloc[ind, 1:] = return_most_common_venues(recommended_group.iloc[ind, :], num_top_venues)

recommended_venues_sorted.head()

,categories,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Lake,Tidal Basin,World War II Memorial,Washington Monument,Vietnam Veterans Memorial,Thomas Jefferson Memorial,"Martin Luther King, Jr. Memorial",Lincoln Memorial Reflecting Pool,Lincoln Memorial,Korean War Veterans Memorial,Franklin Delano Roosevelt Memorial
1,Monument / Landmark,World War II Memorial,Washington Monument,Vietnam Veterans Memorial,Thomas Jefferson Memorial,"Martin Luther King, Jr. Memorial",Lincoln Memorial Reflecting Pool,Lincoln Memorial,Korean War Veterans Memorial,Franklin Delano Roosevelt Memorial,Capitol Reflecting Pool


# clustering of venues based on the category of venues

In [39]:
cluster = len(recommended_venues['categories'].unique())

recommended_clustering = recommended_group.drop('categories', 1)
recommended_clustering.head()
# run k-means clustering
kmeans = KMeans(n_clusters=cluster, random_state=0).fit(recommended_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:15] 

array([0, 1], dtype=int32)

In [40]:
recommended_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

venue_merged = recommended_venues

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
venue_merged = venue_merged.join(recommended_venues_sorted.set_index('categories'), on='categories')

venue_merged.head(11) # check the last column

,id,name,categories,lat,lng,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
1,40ce3c00f964a5201b011fe3,Washington Monument,Monument / Landmark,38.889401,-77.035244,1,World War II Memorial,Washington Monument,Vietnam Veterans Memorial,Thomas Jefferson Memorial,"Martin Luther King, Jr. Memorial",Lincoln Memorial Reflecting Pool,Lincoln Memorial,Korean War Veterans Memorial,Franklin Delano Roosevelt Memorial,Capitol Reflecting Pool
4,4a518853f964a5201bb11fe3,World War II Memorial,Monument / Landmark,38.889377,-77.040516,1,World War II Memorial,Washington Monument,Vietnam Veterans Memorial,Thomas Jefferson Memorial,"Martin Luther King, Jr. Memorial",Lincoln Memorial Reflecting Pool,Lincoln Memorial,Korean War Veterans Memorial,Franklin Delano Roosevelt Memorial,Capitol Reflecting Pool
12,4a500fe1f964a520f2af1fe3,Vietnam Veterans Memorial,Monument / Landmark,38.891262,-77.047707,1,World War II Memorial,Washington Monument,Vietnam Veterans Memorial,Thomas Jefferson Memorial,"Martin Luther King, Jr. Memorial",Lincoln Memorial Reflecting Pool,Lincoln Memorial,Korean War Veterans Memorial,Franklin Delano Roosevelt Memorial,Capitol Reflecting Pool
14,4a4fbaecf964a520a0af1fe3,Lincoln Memorial,Monument / Landmark,38.889267,-77.050166,1,World War II Memorial,Washington Monument,Vietnam Veterans Memorial,Thomas Jefferson Memorial,"Martin Luther King, Jr. Memorial",Lincoln Memorial Reflecting Pool,Lincoln Memorial,Korean War Veterans Memorial,Franklin Delano Roosevelt Memorial,Capitol Reflecting Pool
16,4b65f636f964a5203e0b2be3,Lincoln Memorial Reflecting Pool,Monument / Landmark,38.889491,-77.045649,1,World War II Memorial,Washington Monument,Vietnam Veterans Memorial,Thomas Jefferson Memorial,"Martin Luther King, Jr. Memorial",Lincoln Memorial Reflecting Pool,Lincoln Memorial,Korean War Veterans Memorial,Franklin Delano Roosevelt Memorial,Capitol Reflecting Pool
20,4bc9f0850687ef3bc7d0dacc,"Martin Luther King, Jr. Memorial",Monument / Landmark,38.886087,-77.044124,1,World War II Memorial,Washington Monument,Vietnam Veterans Memorial,Thomas Jefferson Memorial,"Martin Luther King, Jr. Memorial",Lincoln Memorial Reflecting Pool,Lincoln Memorial,Korean War Veterans Memorial,Franklin Delano Roosevelt Memorial,Capitol Reflecting Pool
23,49d69653f964a520ed5c1fe3,Tidal Basin,Lake,38.884193,-77.039314,0,Tidal Basin,World War II Memorial,Washington Monument,Vietnam Veterans Memorial,Thomas Jefferson Memorial,"Martin Luther King, Jr. Memorial",Lincoln Memorial Reflecting Pool,Lincoln Memorial,Korean War Veterans Memorial,Franklin Delano Roosevelt Memorial
30,4a106621f964a520ba761fe3,Thomas Jefferson Memorial,Monument / Landmark,38.881712,-77.036654,1,World War II Memorial,Washington Monument,Vietnam Veterans Memorial,Thomas Jefferson Memorial,"Martin Luther King, Jr. Memorial",Lincoln Memorial Reflecting Pool,Lincoln Memorial,Korean War Veterans Memorial,Franklin Delano Roosevelt Memorial,Capitol Reflecting Pool
32,4a491e86f964a520ffaa1fe3,Korean War Veterans Memorial,Monument / Landmark,38.887790,-77.047472,1,World War II Memorial,Washington Monument,Vietnam Veterans Memorial,Thomas Jefferson Memorial,"Martin Luther King, Jr. Memorial",Lincoln Memorial Reflecting Pool,Lincoln Memorial,Korean War Veterans Memorial,Franklin Delano Roosevelt Memorial,Capitol Reflecting Pool
35,4a10649df964a520b7761fe3,Franklin Delano Roosevelt Memorial,Monument / Landmark,38.883409,-77.042788,1,World War II Memorial,Washington Monument,Vietnam Veterans Memorial,Thomas Jefferson Memorial,"Martin Luther King, Jr. Memorial",Lincoln Memorial Reflecting Pool,Lincoln Memorial,Korean War Veterans Memorial,Franklin Delano Roosevelt Memorial,Capitol Reflecting Pool


# plotting map for recommended places

In [41]:
# create map
geolocator = Nominatim(user_agent = "{}_explorer".format(address))

location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude


recommended_venue_map = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(cluster)
ys = [i + x + (i*x)**2 for i in range(cluster)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, clust in zip(venue_merged['lat'], venue_merged['lng'], venue_merged['categories'], venue_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(clust), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[clust-1],
        fill=True,
        fill_color=rainbow[clust-1],
        fill_opacity=0.7).add_to(recommended_venue_map)
       
recommended_venue_map

In [42]:
venue_merged.loc[venue_merged['Cluster Labels'] == 0, venue_merged.columns[[1] + list(range(5, venue_merged.shape[1]))]]

,name,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
23,Tidal Basin,0,Tidal Basin,World War II Memorial,Washington Monument,Vietnam Veterans Memorial,Thomas Jefferson Memorial,"Martin Luther King, Jr. Memorial",Lincoln Memorial Reflecting Pool,Lincoln Memorial,Korean War Veterans Memorial,Franklin Delano Roosevelt Memorial


In [43]:
venue_merged.loc[venue_merged['Cluster Labels'] == 1, venue_merged.columns[[1] + list(range(5, venue_merged.shape[1]))]]

,name,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
1,Washington Monument,1,World War II Memorial,Washington Monument,Vietnam Veterans Memorial,Thomas Jefferson Memorial,"Martin Luther King, Jr. Memorial",Lincoln Memorial Reflecting Pool,Lincoln Memorial,Korean War Veterans Memorial,Franklin Delano Roosevelt Memorial,Capitol Reflecting Pool
4,World War II Memorial,1,World War II Memorial,Washington Monument,Vietnam Veterans Memorial,Thomas Jefferson Memorial,"Martin Luther King, Jr. Memorial",Lincoln Memorial Reflecting Pool,Lincoln Memorial,Korean War Veterans Memorial,Franklin Delano Roosevelt Memorial,Capitol Reflecting Pool
12,Vietnam Veterans Memorial,1,World War II Memorial,Washington Monument,Vietnam Veterans Memorial,Thomas Jefferson Memorial,"Martin Luther King, Jr. Memorial",Lincoln Memorial Reflecting Pool,Lincoln Memorial,Korean War Veterans Memorial,Franklin Delano Roosevelt Memorial,Capitol Reflecting Pool
14,Lincoln Memorial,1,World War II Memorial,Washington Monument,Vietnam Veterans Memorial,Thomas Jefferson Memorial,"Martin Luther King, Jr. Memorial",Lincoln Memorial Reflecting Pool,Lincoln Memorial,Korean War Veterans Memorial,Franklin Delano Roosevelt Memorial,Capitol Reflecting Pool
16,Lincoln Memorial Reflecting Pool,1,World War II Memorial,Washington Monument,Vietnam Veterans Memorial,Thomas Jefferson Memorial,"Martin Luther King, Jr. Memorial",Lincoln Memorial Reflecting Pool,Lincoln Memorial,Korean War Veterans Memorial,Franklin Delano Roosevelt Memorial,Capitol Reflecting Pool
20,"Martin Luther King, Jr. Memorial",1,World War II Memorial,Washington Monument,Vietnam Veterans Memorial,Thomas Jefferson Memorial,"Martin Luther King, Jr. Memorial",Lincoln Memorial Reflecting Pool,Lincoln Memorial,Korean War Veterans Memorial,Franklin Delano Roosevelt Memorial,Capitol Reflecting Pool
30,Thomas Jefferson Memorial,1,World War II Memorial,Washington Monument,Vietnam Veterans Memorial,Thomas Jefferson Memorial,"Martin Luther King, Jr. Memorial",Lincoln Memorial Reflecting Pool,Lincoln Memorial,Korean War Veterans Memorial,Franklin Delano Roosevelt Memorial,Capitol Reflecting Pool
32,Korean War Veterans Memorial,1,World War II Memorial,Washington Monument,Vietnam Veterans Memorial,Thomas Jefferson Memorial,"Martin Luther King, Jr. Memorial",Lincoln Memorial Reflecting Pool,Lincoln Memorial,Korean War Veterans Memorial,Franklin Delano Roosevelt Memorial,Capitol Reflecting Pool
35,Franklin Delano Roosevelt Memorial,1,World War II Memorial,Washington Monument,Vietnam Veterans Memorial,Thomas Jefferson Memorial,"Martin Luther King, Jr. Memorial",Lincoln Memorial Reflecting Pool,Lincoln Memorial,Korean War Veterans Memorial,Franklin Delano Roosevelt Memorial,Capitol Reflecting Pool
93,Capitol Reflecting Pool,1,World War II Memorial,Washington Monument,Vietnam Veterans Memorial,Thomas Jefferson Memorial,"Martin Luther King, Jr. Memorial",Lincoln Memorial Reflecting Pool,Lincoln Memorial,Korean War Veterans Memorial,Franklin Delano Roosevelt Memorial,Capitol Reflecting Pool
